In [ ]:
pip install --upgrade google-cloud-aiplatform

# Import packages and instantiate models

In [5]:
from vertexai.preview.generative_models import GenerativeModel, GenerationConfig, Image, Part, FunctionDeclaration, Tool
from vertexai.language_models import TextGenerationModel
import vertexai.preview.generative_models as generative_models
import http.client
import typing
import urllib.request

text_bison_model = TextGenerationModel.from_pretrained("text-bison")
gemini_pro_model = GenerativeModel("gemini-1.0-pro")
gemini_pro_vision_model = GenerativeModel("gemini-1.0-pro-vision")


# Example of making a prediction using Palm2 (text-bison)

In [6]:
model_response = text_bison_model.predict(
    "Why do cars have four wheels?", 
    max_output_tokens=128, 
    temperature=0, 
    top_p=1, 
    top_k=5)

print(model_response)


MultiCandidateTextGenerationResponse(text=' Cars have four wheels for several reasons:\n\n1. **Stability:** Four wheels provide a wider base of support for the vehicle, which increases its stability. This is especially important when cornering or driving on uneven surfaces.\n\n2. **Traction:** Four wheels provide more traction than two wheels, which is essential for accelerating, braking, and climbing hills.\n\n3. **Load distribution:** Four wheels help to distribute the weight of the vehicle more evenly, which reduces stress on the suspension and tires.\n\n4. **Maneuverability:** Four wheels allow for greater maneuverability than two wheels, making it easier to turn and park.', _prediction_response=Prediction(predictions=[{'content': ' Cars have four wheels for several reasons:\n\n1. **Stability:** Four wheels provide a wider base of support for the vehicle, which increases its stability. This is especially important when cornering or driving on uneven surfaces.\n\n2. **Traction:** Fo

# Compare making a prediction using Gemini model
Reference to Gemini API (https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini)

In [7]:
config = GenerationConfig(
    # Only one candidate for now.
    candidate_count=1,
    max_output_tokens=128,
    temperature=1.0,
    top_p=1,
    top_k=5)
model_response = gemini_pro_model.generate_content(
    "Why do cars have four wheels?", 
    generation_config=config)

print(model_response)


candidates {
  content {
    role: "model"
    parts {
      text: "Stability and Maneuverability: Four wheels provide a stable base for the vehicle, preventing it from tipping over even when cornering or driving on uneven surfaces. This enhanced stability allows for better control and handling of the car.\n\nWeight Distribution: The weight of the vehicle is distributed evenly across the four wheels, maximizing traction and reducing the load on any single wheel. This helps prevent punctures and ensures a smooth ride.\n\nCornering Ability: Four wheels allow the car to corner more effectively. The outer wheels can grip the road more firmly, generating more cornering force without the risk of losing traction. This improved cornering ability contributes to safe and"
    }
  }
  finish_reason: MAX_TOKENS
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
 

# Multi-line prompting between Palm2 and Gemini is different

In [8]:
model_response = text_bison_model.predict("""What is x multiplied by 2?
x=42""")

print(model_response)


MultiCandidateTextGenerationResponse(text=' To find the value of x multiplied by 2, we simply multiply x by 2. Since x is equal to 42, we have:\n\nx * 2 = 42 * 2 = 84\n\nTherefore, x multiplied by 2 is equal to 84.', _prediction_response=Prediction(predictions=[{'content': ' To find the value of x multiplied by 2, we simply multiply x by 2. Since x is equal to 42, we have:\n\nx * 2 = 42 * 2 = 84\n\nTherefore, x multiplied by 2 is equal to 84.', 'citationMetadata': {'citations': []}, 'safetyAttributes': {'categories': ['Derogatory', 'Finance', 'Health', 'Insult', 'Religion & Belief', 'Sexual'], 'blocked': False, 'safetyRatings': [{'category': 'Dangerous Content', 'severity': 'NEGLIGIBLE', 'severityScore': 0.0, 'probabilityScore': 0.2}, {'severityScore': 0.1, 'category': 'Harassment', 'severity': 'NEGLIGIBLE', 'probabilityScore': 0.1}, {'probabilityScore': 0.1, 'severityScore': 0.1, 'severity': 'NEGLIGIBLE', 'category': 'Hate Speech'}, {'severityScore': 0.1, 'probabilityScore': 0.1, 'sev

# With Gemini, it can be done by passing an array of strings

In [10]:
model_response = gemini_pro_model.generate_content(["What is x multiplied by 2?", "x = 42"])

print(model_response)


candidates {
  content {
    role: "model"
    parts {
      text: "84"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 13
  candidates_token_count: 2
  total_token_count: 15
}



# Both Palm2 and Gemini support streaming but they are invoked differently

In [12]:
prompt = "Tell me the story of Alice in Wonderland"
model_response = text_bison_model.predict_streaming(prompt)

for chunk in model_response:
  print(chunk.text)


 Once upon a time, in a quaint little village, there lived a curious and imaginative girl named Alice.
 One sunny afternoon, while sitting on the riverbank, Alice noticed a peculiar white rabbit wearing a waistcoat and
 a pocket watch. Intrigued, she followed the rabbit down a rabbit hole and found herself in
 a strange and wonderful world known as Wonderland.

In Wonderland, Alice encountered a series of extraordinary characters,
 each with their own unique quirks and personalities. She met the Cheshire Cat, who could vanish and reappear at
 will, leaving behind only a mischievous grin. She also encountered the Mad Hatter, who hosted never-ending tea
 parties,


In [13]:
prompt = "Tell me the story of Alice in Wonderland"
model_response = gemini_pro_model.generate_content(prompt, stream=True)

for chunk in model_response:
  print(chunk.text)


**Chapter 1: Down the Rabbit-Hole**
Alice, a young girl, follows a white rabbit down a rabbit hole and falls into a strange and
 wonderful world.

**Chapter 2: The Pool of Tears**
Alice finds herself in a pool of her own tears, where she meets a talking mouse
, a dodo bird, and other creatures.

**Chapter 3: A Caucus-Race and a Long Tale**
Alice attends a caucus-race, where the animals run in circles and get nowhere. She then meets a caterpillar who gives her a piece of advice: "Who are you?"

**Chapter
 4: Tweedledum and Tweedledee**
Alice encounters the Tweedle twins, who argue about which day is the best and tell a nonsensical poem.

**Chapter 5: The Lobster Quadrille**
Alice attends a lobster quadrille, where she learns to dance with a lobster.

**
Chapter 6: Pig and Pepper**
Alice enters the Duchess's kitchen, where she meets the Duchess, her cook, and her baby. The baby turns into a pig, and Alice escapes from the kitchen.

**Chapter 7: A Mad Tea-Party**
Alice joins the Mad Hatte

# It is possible to adjust safety settings in Gemini as well
Reference the following links for categories and thresholds:
    
* https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.HarmCategory
* https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.SafetySetting.HarmBlockThreshold

In [14]:
prompt = "Tell me the story of Alice in Wonderland"
safety_settings={
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}    

model_response = gemini_pro_model.generate_content(prompt, safety_settings=safety_settings)
print(model_response)
    

candidates {
  content {
    role: "model"
    parts {
      text: "**Chapter 1: Down the Rabbit-Hole**\n\nAlice, a young girl bored on a summer afternoon, follows a white rabbit down a curious rabbit-hole. She falls deep into a strange and nonsensical world called Wonderland.\n\n**Chapter 2: The Pool of Tears**\n\nAlice\'s tears create a pool that floods Wonderland. She meets a quirky Mouse, a Dodo, and other creatures who offer unhelpful advice.\n\n**Chapter 3: A Caucus-Race and a Long Tale**\n\nAlice participates in a chaotic race with the animals, then listens to the Mouse tell a long and incomprehensible tale.\n\n**Chapter 4: The Rabbit Sends in a Little Bill**\n\nAlice enters the Rabbit\'s house and confuses it for her own. She encounters a baby, a talking Caterpillar, and a bottle that makes her shrink and grow.\n\n**Chapter 5: Advice from a Caterpillar**\n\nThe Caterpillar advises Alice to eat a portion of mushroom to change her size. However, the mushroom has different effects

# Multi-modal prompts are supported in Gemini
The following makes a few shot prompt using images. The images are retrieved from public urls and passed to the prompt as an Image object.

In [15]:
def load_image_from_url(image_url: str) -> Image:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return Image.from_bytes(image_bytes)


# Load images from Cloud Storage URI
landmark1 = load_image_from_url(
    "https://storage.googleapis.com/cloud-samples-data/vertex-ai/llm/prompts/landmark1.png")
landmark2 = load_image_from_url(
    "https://storage.googleapis.com/cloud-samples-data/vertex-ai/llm/prompts/landmark2.png")
landmark3 = load_image_from_url(
    "https://storage.googleapis.com/cloud-samples-data/vertex-ai/llm/prompts/landmark3.png")

# Pass multimodal prompt
model_response = gemini_pro_vision_model.generate_content([
    landmark1,
    "city: Rome, Landmark: the Colosseum",
    landmark2,
    "city: Beijing, Landmark: Forbidden City",
    landmark3])

print(model_response)


candidates {
  content {
    role: "model"
    parts {
      text: " city: Rio de Janeiro, Landmark: Christ the Redeemer"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 791
  candidates_token_count: 11
  total_token_count: 802
}



This example uses the `from_uri` method to reference a GCS path. The method only works with GCS paths.

In [16]:
image = Part.from_uri("gs://cloud-samples-data/ai-platform/flowers/daisy/10559679065_50d2b16f6d.jpg", mime_type="image/jpeg")
# https://storage.googleapis.com/cloud-samples-data/ai-platform/flowers/daisy/10559679065_50d2b16f6d.jpg
model_response = gemini_pro_vision_model.generate_content(["what is this image?", image])

print(model_response)


candidates {
  content {
    role: "model"
    parts {
      text: " This is an image of a flower. The flower has white petals and a yellow center. It is surrounded by brown and yellow leaves."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 263
  candidates_token_count: 27
  total_token_count: 290
}



# Videos can also be used in prompts with Gemini

In [17]:
video = Part.from_uri("gs://cloud-samples-data/video/animals.mp4", mime_type="video/mp4")
# https://storage.googleapis.com/cloud-samples-data/video/animals.mp4

model_response = gemini_pro_vision_model.generate_content(["What phone is featured in the video?", video])
print(model_response.text)

model_response = gemini_pro_vision_model.generate_content(["What animals are shown in the video?", video])
print(model_response.text)


 The phone featured in the video is the Google Pixel.
 The video shows a sloth, a fox, a rabbit, a tiger, an elephant, a seal, an otter, and a beaver.


# Function calling is a feature only available on Gemini 
It returns a structured data object with a function name and parameters from the prompt that allows a developer to construct an API call

In [18]:
get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given city",
    parameters={
        "type": "object",
        "properties": {"location": {"type": "string", "description": "City"}},
    },
)

get_restaurants_func = FunctionDeclaration(
    name="get_restaurants",
    description="Get the restaurants in a given neighborhood in a given city",
    parameters={
        "type": "object",
        "properties": {
            "neighborhood": {"type": "string", "description": "Neighborhood"},
            "city": {"type": "string", "description": "City"}
        },
    }
)

city_tool = Tool(
    function_declarations=[get_current_weather_func, get_restaurants_func]
)


In [19]:
model_response = gemini_pro_model.generate_content(
    "What is the weather like in Boston?", 
    generation_config={"temperature": 0}, 
    tools=[city_tool])

print(model_response)


candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_current_weather"
        args {
          fields {
            key: "location"
            value {
              string_value: "Boston"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 41
  candidates_token_count: 7
  total_token_count: 48
}



In [20]:
model_response = gemini_pro_model.generate_content(
    "Where are some good restaurants in Old City Philadelphia?",
    generation_config={"temperature": 0},
    tools=[city_tool])
function_call = model_response.candidates[0].content.parts[0].function_call

print(function_call)


name: "get_restaurants"
args {
  fields {
    key: "city"
    value {
      string_value: "Philadelphia"
    }
  }
  fields {
    key: "neighborhood"
    value {
      string_value: "Old City"
    }
  }
}



In [21]:
# example of how you might handle the response
if (function_call.name == "get_restaurants"):
    params = {}
    for key, value in function_call.args.items():
        params[key] = value
    print(params)
    
    # response = requests.post(get_restaurants_url, data: params)



{'neighborhood': 'Old City', 'city': 'Philadelphia'}


In [22]:
# will not handle anything outside the scope of the functions
model_response = gemini_pro_model.generate_content(
    "Why is the sky blue?",
    generation_config={"temperature": 0},
    tools=[city_tool])

print(model_response)


candidates {
  content {
    role: "model"
    parts {
      text: "I cannot fulfill this request. I do not have the functionality to answer questions about the sky."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 39
  candidates_token_count: 19
  total_token_count: 58
}

